# Change the TARA results format to write the AEP steady state report.
## Section 1 need to be run first for all of the rest sections.
### 1. Define a function to get the bus name and information in the Planning data dictionary.

In [1]:
"""This section defines a function to get the bus info given the number or the sswg name."""
import numpy as np
import pandas as pd

# Read the excel file
df = pd.read_excel("data/ERCOT_24SSWGU1_Planning_Data_Dictionary_11072024.xlsx",sheet_name=2,usecols="B,C,F,R")

# Define a function get the information of the bus
def get_bus_info(bus_number, sswg_name=None):
    """Input the bus number (int), return the SSWG bus name, bus voltage, and the planned longname"""

    bus = np.logical_or(df['SSWG BUS NUMBER'].values == bus_number,df['SSWG BUS NAME'].values == sswg_name)
    df_bus = df.loc[bus]
    try:
        if df_bus.iloc[0]['SSWG BASE KV'] % 1 == 0:
            return df_bus.iloc[0]['SSWG BUS NAME'], df_bus.iloc[0]['PLANNING BUS LONGNAME'].title(), int(df_bus.iloc[0]['SSWG BASE KV'])
        else:
            return df_bus.iloc[0]['SSWG BUS NAME'], df_bus.iloc[0]['PLANNING BUS LONGNAME'].title(), df_bus.iloc[0]['SSWG BASE KV']
    except IndexError:
        print(f'Bus {bus_number} is not in the planning data dictionary.')


Test the function.

In [2]:
get_bus_info(5725)

('PAWNEESW5', 'Pawneesw5', 345)

### 2. Define the function to input the TARA voltage violation bus cell and output the voltage violation result as a sentence in report.

In [69]:
"""This section defines a function to convert the bus number to long bus name for bus in report VOLTAGE violation table."""
def volviolation(bus):
    import re

    bus = bus.strip()
    bus = re.sub(r'\s+', ' ', bus)
    bus = bus.rstrip(' kV')

    # Get the bus number, sswg name and voltage
    num = re.findall(r'(\d+) ', bus)[0]
    sswg_name = re.findall(r' (\w+)', bus)[0]
    volt = re.findall(r'\d+$',bus)[0]
    try:
        [sswg_name, long_name, bus_voltage] = get_bus_info(int(num),sswg_name)
        if long_name[-2:] == 'kV':
            long_name = long_name[:-2]
            #print(f' the {long_name.title()} kV bus,')
            return (f'The {long_name.title()} kV bus begins to experience')
        else:
            #print(f' the {long_name.title()} ' + volt + ' kV bus,')
            return (f'The {long_name.title()} {bus_voltage} kV bus begins to experience')
    except TypeError:
        return ('Write this bus voltage violation manually to the report.')

Test the function.

In [4]:
# User input the Bus description
# bus = input("Please enter the bus in the report table: ")
# bus = '44343 IMPERL05T3_8 138'
bus = '8732 DEVINE4A    138'
print(volviolation(bus))

The Devine 138 kV bus begins to experience large voltage drop for the loss of 


### 3. Define the function to input the TARA thermal violation branch cell and output the thermal violation result as a sentence in report.

In [18]:
"""This section defines a function to convert the bus number to long bus name for branch in report THERMAL violation table."""
def thmviolation(branch):
    import re
    branch = branch.strip()
    branch = re.sub(r'\s+', ' ', branch)
    branch = branch.replace('.0','')

    # Get two bus numbers and voltage
    info = re.findall(r'(\d+) \D', branch)[:3]
    volt = re.findall(r' \d+ ',branch)
    ckt = re.findall(r'\w+$',branch)[0]

    bus_name = []
    try:
        for i in range(2):
            [sswg_name, long_name, bus_voltage] = get_bus_info(int(info[i]),'')
            bus_name.append(long_name.title())

        if volt[0] == volt[1]:  # AC line
            return (f"The {bus_name[0]} to {bus_name[1]}"+ volt[0] +"kV line circuit " + ckt + ' begins to experience overloading for the loss of ')
        else:   # Transformer
            return (f"The {bus_name[0]}{volt[0]}kV to {bus_name[1]}"+ volt[1] +'kV transformer begins to experience overloading for the loss of ')
    except TypeError:
        return ('Write this branch thermal violation manually to the report.')

Test the function.

In [57]:
# User input the Branch description
# branch = input("Please enter the branch in the report table: ")
# Test
# branch = '8186 KENEDYSW4A    138    8913 KARNES4A      138  1  '
# branch = '8186 KENEDYSW4A 138 8913 KARNES4A 138  1  '
# branch = '5400 SPRUCE 345 5295 MARTINEZ_5 345 1'          # Error because 5295 not in the planning data dictionary
# branch = '110015 WAP_WAP_G5 20.0 49216 GENWAP5_05_8 138 G1'
# branch = '49216 GENWAP5_05_8 138 44010 W_A_P__POI8A 138 GL'
branch = ' 7244 L_CUERO_8_1Y 138 89450 THOMASTN4B 138 1'
print(thmviolation(branch))

The Cuero Lcra to Thomaston Pst Load Side 138 kV line circuit 1 begins to experience overloading for the loss of 


### 4. Define the function to input the Contigency ID and output the contigency legend or description in report.

In [6]:
"""This section is used to convert contingency to two different forms in reports. The year and season are needed."""
"""Run the prepare_file jupyter notebook to get the all-together contigency file."""
def get_con(id, year, season):
    import re

    season = season.strip().upper()
    id = id.rstrip().upper()

    contingency = ''

    # Find the contingency in the all-together file
    file_name = 'data/'+year[2:]+season+'_All.con'
    with open (file_name, encoding="utf8", errors='ignore') as file:
        lines = file.readlines()

        # looping through each line in the file 
        for line in lines: 
            
            # if line have the input string, get the index of that line  
            if '\''+id+'\'' in line: 
                idx = lines.index(line)
                

                # find where this contingency end
                while ' END' not in lines[idx]:
                    contingency = contingency + lines[idx] #+ "\n"
                    idx += 1
                break
        # closing file after reading 
        file.close() 

    # Create the contingency for Table legend
    cont_0 = contingency.splitlines()[1:]
    cont_1 = contingency.splitlines()[1:]
    cont_2 = contingency.splitlines()[1:]
    for line in range(len(cont_1)):
        cont_0[line] = cont_0[line].split('/',1)[0].lstrip().rstrip()
        cont_1[line] = cont_1[line].split('/',1)[0].lstrip().rstrip().lower()
        cont_2[line] = cont_2[line].split('/',1)[0].lstrip().rstrip().lower()

        bus_list = re.findall(r'bus (\d+)', cont_1[line])
        for i in range(len(bus_list)):
            bus_number = int(bus_list[i])
            [sswg_name, long_name, bus_voltage] = get_bus_info(bus_number,'')
            if 'open bus' in cont_1[line]:
                cont_1[line] = cont_1[line].replace(f'{bus_list[i]}',f'{bus_list[i]} [{sswg_name} {bus_voltage}]')
            else:
                cont_1[line] = cont_1[line].replace(f'bus {bus_list[i]}',f'{bus_list[i]} [{sswg_name} {bus_voltage}]')
            if i == len(bus_list)-1:    # the last bus
                if len(bus_list) == 2:
                    cont_2[line] = cont_2[line].replace(f'bus {bus_list[i]}',f'{long_name.title()} {bus_voltage} kV line')
                else:
                    cont_2[line] = cont_2[line].replace(f'bus {bus_list[i]}',f'{long_name.title()} {bus_voltage} kV bus')
            else:
                cont_2[line] = cont_2[line].replace(f'bus {bus_list[i]}',f'{long_name.title()}')
            cont_2[line] = cont_2[line].replace('open branch from','the')
            cont_2[line] = cont_2[line].replace('Kv','kV')
            cont_2[line] = cont_2[line].replace('ckt','circuit')
            cont_2[line] = cont_2[line].replace('open','the')
            cont_2[line] = cont_2[line].replace('remove','the')     

    # Change cont2 from line lists to a sentence
    cont_2_sen = ''
    for i in range(len(cont_2)):
        if i > 0 and cont_2[i][:-1] == cont_2[i-1][:-1]:
                cont_2_sen = cont_2_sen[:-2]+' and 2, '
        else:
            if len(cont_2) != 1 and i == len(cont_2)-1:
                cont_2_sen += 'and '+cont_2[i] +','
            else:
                cont_2_sen = cont_2_sen + cont_2[i] + ', '
    return cont_1, cont_2_sen

# Define a function to print out the contingency in lines
def print_con(cont):
    for line in range(len(cont)):
        print(cont[line])

# Define a function to create the contingency legend
def get_conLegend(id, year, season):
    print_con(get_con(id, year, season)[0])

def get_conDescription(id, year, season):
    return (get_con(id, year, season)[1])
    

Test the functions.

In [66]:
"""Test different contingency."""
# id = input("Please enter the contingency ID you want to convert (DB_ID_XXXXX): ")
# id = 'DB_ID_7499'
# id = 'DB_ID_633'        # remove unit from bus
# id = 'DB_ID_26591'     # remove branch multiple lines
# id = 'DB_ID_38746'     # remove bus, remove branch combined
# id = 'DB_ID_50794'    # double circuit same branch name
id = 'DB_ID_12867'
# id = 'DB_ID_7728'
# get_conLegend(id,'2025','spg')
print(get_conDescription(id,'2028','sum'))

the Riverside Pump 69 kV bus, 


### 5. Define the function to convert one line in the result table to a paragraph.

In [ ]:
def get_paragraph(scenario, project_name, year, season, copied_texts):

    import re

    copied_texts = copied_texts.rstrip()
    copied_texts = copied_texts.replace('.0','')
    splitted_texts = copied_texts.split('	')

    # Get the results
    trans_limits = splitted_texts[:4]
    item = splitted_texts[4]
    id = splitted_texts[5]
    id = re.sub(r'\w+_\w+_\d+ & ', '', id)

    # existing gen results
    res = 'when '+project_name+' is '
    if scenario == 'a':
        res += 'discharging'
    elif scenario == 'b':
        res += 'charging'

    left, right = 0, 3
    level = [25,50,75,100]

    # Iterate while left pointer is less than right
    while left < right:
        if trans_limits[left][0] != '>' and trans_limits[right][0] != '>':
            break
        elif trans_limits[left][0] == '>': 
            left += 1  # Move left pointer to the right
        else:
            right -= 1 # Move right pointer to the left
    if left != 0 and right != 3:
        print('This result does not make sense, please double check!')
    elif left == right or trans_limits[left] == trans_limits[right]:
        res += f' at {trans_limits[left]} MW'
    elif scenario == 'a':
        res += f' from {trans_limits[left]} MW to {trans_limits[right]} MW'
    else:
        res += f' from {trans_limits[right]} MW to {trans_limits[left]} MW'

    res += ' and existing area generation is dispatched'
    if left == right:
        res += f' at {level[left]}% in {year}'
    elif scenario == 'a':
        res+= f' from {level[left]}% to {level[right]}% in {year}'
    else:
        res+= f' from {level[right]}% to {level[left]}% in {year}'

    if season == 'sum':
        res += ' Summer Case.'
    elif season == 'min':
        res += ' Minimum Case.'
    elif season == 'spg':
        res += ' Spring Case.'
    else:
        print('Season information is incorrect!')
        return True
    # Output the report description
    if splitted_texts[-1] == '100':
        return (thmviolation(item)+get_conDescription(id,year,season)+res)
    elif splitted_texts[-1] == '0.92':
        return (volviolation(item)+' low voltage for the loss of '+get_conDescription(id,year,season)+res)
    else:
        return (volviolation(item)+' large voltage drop for the loss of '+get_conDescription(id,year,season)+res)



In [74]:
scenario = 'a'
project_name = 'Chacon Creek BESS'
year = '2028'
season = 'min'

# copied_texts = input('Paste')
# copied_texts = '>160.88	>160.88	>160.88	106.7	11048 TONKAWAW_5    345   10049 RANGER_5      345  1	DB_ID_11020	21.3	1072	100.0'
# copied_texts = '>204.8	0	0	0	  5725 PAWNEESW5     345    5400 SPRUCE        345  1  	DB_ID_2017	45.82	1058	100.0'
# copied_texts = '>160.88	>160.88	93.5	0	11048 TONKAWAW_5    345   10049 RANGER_5      345  1	DB_ID_47531	21.3	1072	100.0'
# copied_texts = '>160.88	57.8	0	0	   11306 DERMOTT_8     138    1312 SCURYCHV_8    138  1  	DB_ID_51092 & DB_ID_46789	8.4	394	100.0'
# copied_texts = '>181.9	116.6	0	0	  5422 TALLY_RD      138    7432 L_MEDILA8_1Y  138  1  	DB_ID_28198	12.4	220	100.0'
copied_texts = '0	0	0	0	8732 DEVINE4A    138	DB_ID_7695	0.92'

print(get_paragraph(scenario, project_name, year, season, copied_texts))


The Devine 138 kV bus begins to experience low voltage for the loss of the Lytle to Lytle 138 kV line circuit 1, when Chacon Creek BESS is discharging at 0 MW and existing area generation is dispatched from 25% to 100% in 2028 Minimum Case.
